# Advanced RAG configuration with LangChain

- Author: Martin Fockedey with the help of copilot
- This is based on [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)


## Environment Setup

In [2]:
%pip install -q python-dotenv langchain_mistralai langchain_text_splitters langchain_community langchain_core faiss-cpu pymupdf rank_bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\FKY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# Load environment variables (expects MISTRAL_API_KEY in .env)
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Load & Inspect Document
We load the ECAM regulation PDF (`RéglementECAM.pdf`). Make sure the file is placed in the same working directory or adjust the path.


In [ ]:
# Optional dependency for WebBaseLoader examples
%pip install -q bs4

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader
import os
PDF_PATH = 'RéglementECAM.pdf'
assert os.path.exists(PDF_PATH), f'File not found: {PDF_PATH}'
loader = PyMuPDFLoader(PDF_PATH)
docs = loader.load()
len(docs), docs[0].metadata

(61,
 {'source': 'RéglementECAM.pdf',
  'file_path': 'RéglementECAM.pdf',
  'page': 0,
  'total_pages': 61,
  'format': 'PDF 1.7',
  'title': 'Vade Mecum',
  'author': 'MD',
  'subject': '',
  'keywords': '',
  'creator': 'Microsoft® Word pour Microsoft\xa0365',
  'producer': 'Microsoft® Word pour Microsoft\xa0365',
  'creationDate': "D:20250912135321+02'00'",
  'modDate': "D:20250912135321+02'00'",
  'trapped': ''})

In [5]:
# Preview a page (adjust index as needed)
page_index = 5
print(docs[page_index].page_content[:800])
print('Metadata:', docs[page_index].metadata)

Haute Ecole ICHEC – ECAM – ISFSC - Règlement des études 2025-2026 
6 
 
vue de s'assurer que l’étudiante ou l’étudiant a acquis les matières prérequises pour les 
études visées ; 
e. un grade académique similaire à ceux mentionnés aux littéras précédents délivré par un 
établissement d'enseignement supérieur, en Communauté française ou extérieur à celle-
ci, en vertu d'une décision des autorités académiques et aux conditions complémentaires 
qu'elles fixent en vue de s'assurer que l’étudiante ou l’étudiant a acquis les matières 
prérequises pour les études visées ; 
f. 
Est similaire à un grade académique délivré en Communauté française, un titre ou grade 
conduisant aux mêmes capacités d'accès professionnel ou de poursuite d'études dans le 
système d'origine. 
g. un grade académique étran
Metadata: {'source': 'RéglementECAM.pdf', 'file_path': 'RéglementECAM.pdf', 'page': 5, 'total_pages': 61, 'format': 'PDF 1.7', 'title': 'Vade Mecum', 'author': 'MD', 'subject': '', 'keywords': '', 'c

### Other Common Loaders (quick overview)
It's not only pdf files that you can load but also txt,code files or directly web pages


Below are frequently used loaders you can swap in depending on your source type. They mirror the options shown in the advanced tutorial:

- Web pages (`WebBaseLoader`) – parse specific sections with BeautifulSoup. Note: set a `USER_AGENT` in your `.env` if required by the site.
```python
from langchain_community.document_loaders import WebBaseLoader
import bs4
url = "https://www.bbc.com/news/business-68092814"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("main", attrs={"id": ["main-content"]}))
)
docs = loader.load()
```

- PDF – This notebook uses `PyMuPDFLoader` (fast, robust). You can also use `PyPDFLoader`:
```python
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("path/to/file.pdf")
docs = loader.load()
# OR
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("path/to/file.pdf")
docs = loader.load()
```

- CSV (`CSVLoader`) – Rows become documents:
```python
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="data/titanic.csv")
docs = loader.load()
```

- TXT (`TextLoader`) – Plain text files:
```python
from langchain_community.document_loaders import TextLoader
loader = TextLoader("data/appendix-keywords_eng.txt", encoding="utf-8")
docs = loader.load()
```

- Directory (`DirectoryLoader`) – Load many files with a glob pattern:
```python
from langchain_community.document_loaders import DirectoryLoader
# All TXT files in a folder
loader = DirectoryLoader(".", glob="data/*.txt", show_progress=True)
docs = loader.load()

# All PDF files in a folder
loader = DirectoryLoader(".", glob="data/*.pdf")
docs = loader.load()
```

- Python files – Use `DirectoryLoader` + `PythonLoader`:
```python
from langchain_community.document_loaders import DirectoryLoader, PythonLoader
loader = DirectoryLoader(".", glob="**/*.py", loader_cls=PythonLoader)
docs = loader.load()
```

## Split Documents


### CharacterTextSplitter

This is the simplest method. It splits the text based on characters (default: "\n\n") and measures the chunk size by the number of characters.

1. **How the text is split** : By single character units.
2. **How the chunk size is measured** : By the ```len``` of characters.

Visualization example: https://chunkviz.up.railway.app/


The ```CharacterTextSplitter``` class provides functionality to split text into chunks of a specified size.

- ```separator``` parameter specifies the string used to separate chunks, with two newline characters ("\n") being used in this case.
- ```chunk_size```determines the maximum length of each chunk. When this is too small you obtain too many short documents that don't contain information individually and when too big the embeddings might not be precise enough for the vectore store search.
- ```chunk_overlap```specifies the number of overlapping characters between adjacent chunks.
- ```length_function```defines the function used to calculate the length of a chunk, with the default being the ```len``` function, which returns the length of the string.
- ```is_separator_regex```is a boolean value that determines whether the ```separator``` is interpreted as a regular expression.


In [30]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)
split_docs = text_splitter.split_documents(docs)
len(split_docs),  split_docs[110].metadata, split_docs[110].page_content


(490,
 {'source': 'RéglementECAM.pdf',
  'file_path': 'RéglementECAM.pdf',
  'page': 10,
  'total_pages': 61,
  'format': 'PDF 1.7',
  'title': 'Vade Mecum',
  'author': 'MD',
  'subject': '',
  'keywords': '',
  'creator': 'Microsoft® Word pour Microsoft\xa0365',
  'producer': 'Microsoft® Word pour Microsoft\xa0365',
  'creationDate': "D:20250912135321+02'00'",
  'modDate': "D:20250912135321+02'00'",
  'trapped': ''},
 '▪ \nEt un nombre maximum d’inscriptions dans le cycle concerné. \n \nCes conditions de réussite académique suffisantes, à respecter tout au long du \ncycle d’études concerné, sont impactées par différents critères complémentaires, \nqui tiennent compte du parcours de l’étudiant dans l’enseignement supérieur, \ncomme, par exemple, le bénéficie ou non d’une réorientation ou le respect de \nbalises intermédiaires en termes de valorisation ou d’acquisition de crédits.')

### RecursiveTextSplitter
This text splitter is recommended for general text.

1. ```How the text is split``` : Based on a list of separators.
2. ```How the chunk size is measured``` : By the len of characters.

It has an ordered list of separators (paragraph, newline, space, character). It tries larger separators first to keep bigger coherent units; if resulting pieces are still too large it recursively applies finer separators until each chunk fits chunk_size. Produces more semantically natural chunks.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
custom_separators = [
    "\n \n",        # paragraphs
    "\n",         # lines
    ". ",         # sentence-ish boundary
    "; ",         # clause boundary
    ", ",         # phrase boundary
    " ",          # words
    ""            # fallback: characters
]
text_splitter = RecursiveCharacterTextSplitter(separators=custom_separators,chunk_size=1000, chunk_overlap=80)
split_docs = text_splitter.split_documents(docs)
len(split_docs),  split_docs[110].metadata, split_docs[110].page_content


(316,
 {'source': 'RéglementECAM.pdf',
  'file_path': 'RéglementECAM.pdf',
  'page': 18,
  'total_pages': 61,
  'format': 'PDF 1.7',
  'title': 'Vade Mecum',
  'author': 'MD',
  'subject': '',
  'keywords': '',
  'creator': 'Microsoft® Word pour Microsoft\xa0365',
  'producer': 'Microsoft® Word pour Microsoft\xa0365',
  'creationDate': "D:20250912135321+02'00'",
  'modDate': "D:20250912135321+02'00'",
  'trapped': ''},
 "§3 PROGRAMME DES ETUDIANTS EN FIN DE PREMIER CYCLE \nL’étudiante ou l’étudiant qui doit encore acquérir ou valoriser 15 crédits au maximum du \nprogramme d’études de premier cycle peut compléter son programme annuel avec des unités \nd'enseignement du cycle d'études suivant pour lesquelles elle ou il remplit les conditions \nprérequises, sans que l’ensemble des crédits n’excède 60 crédits. Lorsque les unités \nd’enseignement de Bachelier et de Master sont suivies dans des établissements différents, il \nappartient à l’étudiante ou à l’étudiant de veiller au respect de 

## Embeddings 
- [Link to official documentation - Embedding](https://python.langchain.com/docs/integrations/text_embedding)

Use `MistralAIEmbeddings` (model: `mistral-embed`).  
Embeddings are vector representations of text (tokens, sentences, docs) produced by specialized models closely related to LLMs but optimized for similarity search, clustering, retrieval, and re-ranking rather than generative output.  
Providers offer different embedding models (Mistral, OpenAI, Cohere, Hugging Face, etc.) that vary in dimensionality, cost, speed, and multilingual quality.  

In [33]:
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(model='mistral-embed')

## Vector Store: Build & Persist
We build a FAISS vector store. To avoid recomputation on subsequent runs, we save it locally and load if already present Directory: `faiss_store/` (creates `index.faiss` + `index.pkl`).
Since embedding large corpora consumes tokens (and may incur API cost), generate them once, persist locally (e.g. FAISS index + metadata), and reload instead of recomputing each run. This reduces latency and spend, and enables cheaper iterative experimentation on retriever strategies without re-embedding.

In [35]:
from langchain_community.vectorstores import FAISS
STORE_DIR = 'faiss_store'
if os.path.isdir(STORE_DIR):
    print('Loading existing FAISS store...')
    vectorstore = FAISS.load_local(STORE_DIR, embeddings, allow_dangerous_deserialization=True)
else:
    print('Creating new FAISS store and saving...')
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    vectorstore.save_local(STORE_DIR)

Loading existing FAISS store...


### Why Persist?
Persisting large regulation embeddings saves time & cost (especially when using paid or rate-limited APIs)

## Retriever Strategies

A Retriever is an interface that returns documents when given an unstructured query.

The Retriever does not need to store documents; it only returns (or retrieves) them. To do so when using a retriever with a vectorstore the retriever will find vectors that are "close" to the query vector, but multiple strategies are possible depending on the metric you use to define proximity, thresholds or the number of documents you want to get. 

- [Link to official documentation - Retriever](https://python.langchain.com/docs/integrations/retrievers/)

The **Retriever** is created by using the ```invoke()``` method on the generated VectorStore.

### Strategies and quick metrics

- Similarity: returns the top-k most similar chunks by cosine similarity in the embedding space. Cosine similarity is like a dot product where only the angle between the vectors matters and not their magnitude.
- Similarity + score threshold: filters out low-confidence matches (that have a similarity score lower than a threshold).
- MMR (Maximal Marginal Relevance) selects documents that maximize similarity to the query while minimizing redundancy among the selected documents. [A link for more info](https://medium.com/@ankitgeotek/mastering-maximal-marginal-relevance-mmr-a-beginners-guide-0f383035a985) 
- Ensemble (BM25 + dense): combines lexical and semantic signals for robust retrieval.

In [38]:
# Basic similarity retriever
similarity_retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 4})
query = 'Quelle est la punition pour le plagiat académique?'
basic_results = similarity_retriever.invoke(query)
for i, d in enumerate(basic_results, 1):
    print(f'[{i}] Page={d.metadata.get("page")}, Snippet={d.page_content}')

[1] Page=48, Snippet=En outre, lorsque l’étudiante ou l’étudiant reprend, dans un travail, tout ou partie d’un travail 
personnel ou collectif produit antérieurement, elle ou il veillera à mentionner les références de 
son propre travail ou du travail collectif, sous peine de s’exposer aux sanctions prévues à l’article 
suivant. 
L’essentiel est de pouvoir toujours déterminer quel est l’auteur d’un texte, d’un visuel ou 
audiovisuel, ainsi que d’un élément sonore ou interactif, et de pouvoir distinguer ce qui provient 
d’un tiers ou de ce dont l’étudiante ou l’étudiant est auteur. 
En cas de plagiat avéré, le jury n’aura pas à apporter la preuve de l’intention de frauder. 
 
Toute citation devra être placée entre guillemets et être assortie de la référence précise de la 
source. 
 
Les normes de citation des sources en vigueur dans chaque département sont mises à la 
disposition de l’étudiante ou de l’étudiant, selon les modalités internes au département.
[2] Page=44, Snippet=A l’issue

In [43]:
# Similarity score threshold retriever
threshold_retriever = vectorstore.as_retriever(search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.76, 'k': 6})
threshold_results = threshold_retriever.invoke(query)
print('Results (score >= 0.76):', len(threshold_results))
for i, d in enumerate(threshold_results, 1):
    print(f'[{i}] Page={d.metadata.get("page")}, Snippet={d.page_content}')

Results (score >= 0.76): 2
[1] Page=48, Snippet=En outre, lorsque l’étudiante ou l’étudiant reprend, dans un travail, tout ou partie d’un travail 
personnel ou collectif produit antérieurement, elle ou il veillera à mentionner les références de 
son propre travail ou du travail collectif, sous peine de s’exposer aux sanctions prévues à l’article 
suivant. 
L’essentiel est de pouvoir toujours déterminer quel est l’auteur d’un texte, d’un visuel ou 
audiovisuel, ainsi que d’un élément sonore ou interactif, et de pouvoir distinguer ce qui provient 
d’un tiers ou de ce dont l’étudiante ou l’étudiant est auteur. 
En cas de plagiat avéré, le jury n’aura pas à apporter la preuve de l’intention de frauder. 
 
Toute citation devra être placée entre guillemets et être assortie de la référence précise de la 
source. 
 
Les normes de citation des sources en vigueur dans chaque département sont mises à la 
disposition de l’étudiante ou de l’étudiant, selon les modalités internes au département.
[2]

## MMR Explanation
MMR balances relevance vs diversity, it selects the next document (doc) by maximizing the function:\
 $λ*sim(doc, query) - (1-λ)*max(sim(doc, selected\_docs))$\
 This help to reduce redundant chunks.

In [48]:
# MMR retriever (k=4)
mmr_retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 4, 'fetch_k': 20})
# we didn't find a good query for MMR in this case because we have one document that is not redundant, 
# but imagine a list of close documents that contain similar information
query = "Suis-je financable en tant qu'étudiant international?"
mmr_results = mmr_retriever.invoke(query)
for i, d in enumerate(mmr_results, 1):
    print(f'[MMR {i}] Page={d.metadata.get("page")}, Snippet={d.page_content}')

[MMR 1] Page=11, Snippet=Pour l’étudiante ou l’étudiant dans une autre situation que celle évoquée ci-dessus, 
l’établissement l’avise de sa non-finançabilité et l’invite à motiver sa demande d’inscription ainsi 
qu’à compléter le tableau reprenant son parcours académique. 
 
Pour être recevable, cette demande motivée doit être :   
➢ Formulée sur une seule page A4, ; 
➢ Accompagnée du tableau reprenant son parcours académique dans 
l’enseignement supérieur, disponible sur le site de la Haute Ecole (https://he-
ichec-ecam-isfsc.be/) ; 
➢ Envoyée, par mail, dans un délai de 5 jours à partir de la notification de sa 
non-finançabilité, à l’adresse suivante :
[MMR 2] Page=57, Snippet=Seront toutefois déduits de cette somme, les frais administratifs versés par l’étudiante ou l’étudiant HUE qui ne réside pas sur le territoire 
européen, tels que réclamés dans le cadre de l’instruction de son dossier d’admission (180,00€). 
 
Etudiants libres (article 21 du présent règlement) 
Nombre de créd

## Multi-Query Diversification
Generate paraphrased queries via LLM (Mistral) to expand coverage, then merge unique docs.

In [50]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_mistralai import ChatMistralAI
llm_for_queries = ChatMistralAI(model='mistral-small-latest', temperature=0)
multi_query_retriever = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm_for_queries)
multi_results = multi_query_retriever.get_relevant_documents(query)
print('Unique diversified docs:', len(multi_results))
for i, d in enumerate(multi_results[:5], 1):
    print(f'[Multi {i}] Page={d.metadata.get("page")}, Snippet={d.page_content[:100]}')

Unique diversified docs: 12
[Multi 1] Page=48, Snippet=L’étudiante ou l’étudiant pourra se voir imposer le dépôt de la version électronique de son travail.
[Multi 2] Page=40, Snippet=Ce travail consiste, entre autres, en la rédaction d'un document écrit.  
Le sujet du travail de fin
[Multi 3] Page=11, Snippet=Pour l’étudiante ou l’étudiant dans une autre situation que celle évoquée ci-dessus, 
l’établissemen
[Multi 4] Page=37, Snippet=Le dossier VAE complet doit être introduit auprès du/de la conseiller(e) VAE pour le 31 août au 
plu
[Multi 5] Page=12, Snippet=d’études en fonction du statut dans lequel elle ou il se trouve : 
• 
Etudiant européen : boursier/d


## Ensemble Retriever
Some methods don't use the embeddings and vector distances but still use the words contained in the documents. This is usefull when embeddings fail to represent the document accuratly like when:
    - Queries contain specific keywords, IDs, numbers, dates, names, citations, or jargon (e.g., article codes).
    - Exact wording matters (policies/regulations, legal text, error codes).
    - Out-of-domain or rare vocabulary where embeddings may be weaker.
To do so you can use the [BM25 ranking function](https://en.wikipedia.org/wiki/Okapi_BM25) that uses how unique the words that appear in the documents are and how frequently they appear in different documents while using the [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
It rewards exact token overlap, with:
        - Term frequency saturation (extra repeats help, but with diminishing returns).
        - Inverse document frequency (rare terms get higher weight).
        - Document length normalization (prevents long docs from dominating).

This algorithm has Limitations
    - Does not capture paraphrases or semantic similarity without shared tokens.
    - Synonyms, reworded clauses, or cross-lingual phrasing are missed.
    - Sensitive to tokenization, accents, and hyphenation.

To mitigate the limitation we combine it with dense retrieval (embeddings)

In [66]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# Prepare plain texts for BM25
texts_for_bm25 = [d.page_content for d in split_docs]
bm25 = BM25Retriever.from_texts(texts_for_bm25)
bm25.k = 4
dense_retriever = vectorstore.as_retriever(search_kwargs={'k': 4})
query = "Qui est Frédéric Beaupère ?"
print('--- Without Ensemble Retriever ---')
no_ensemble_results = dense_retriever.invoke(query)
for i, d in enumerate(no_ensemble_results[:2], 1):
    print(f'[WithoutEnsemble {i}] Snippet={d.page_content}')
print('--- Using Ensemble Retriever ---')
ensemble = EnsembleRetriever(retrievers=[bm25, dense_retriever], weights=[0.8, 0.2])
ensemble_results = ensemble.invoke(query)
for i, d in enumerate(ensemble_results[:2], 1):
    print(f'[Ensemble {i}] Snippet={d.page_content}')

--- Without Ensemble Retriever ---
[WithoutEnsemble 1] Snippet=Au terme de cette procédure, la Direction du Département confirme ou non le refus d’inscription 
et, le cas échéant, transmet, au Commissaire du Gouvernement, le nom, le prénom et le sexe 
des auteurs reconnus d’une fraude, de même que la date, le lieu, le pays de naissance de ceux-
ci et l’année académique de la fraude et, s’il échet, leur numéro de registre national ou, à défaut, 
leur numéro d’identification à la Banque Carrefour de la Sécurité sociale.   
Ce dernier, après vérification du respect de la procédure et de la réalité de la fraude, inscrit sans 
délai les informations reçues au sein de la plateforme e-paysage.  L’effacement des fraudeurs 
de la liste se fait après une période de 3 années académiques.
[WithoutEnsemble 2] Snippet=Philippe Dekimpe 
Marie-Françoise Lefebvre 
Master en sciences de 
l’ingénieur industriel – 
orientation construction 
 
Philippe Dekimpe 
Marie-Françoise Lefebvre 
Master en sciences 

## Prompt Engineering
Nous définissons un prompt pour un Assistant Scolaire ECAM (RAG) qui :
- Utilise STRICTEMENT le contexte fourni (pas d'invention)
- Cite les pages source sous la forme `[Page X]` dans le corps et récapitule en fin `CITES: Page: X,Y,...`
- Répond en français, de manière concise et factuelle
- Renvoie "Je ne sais pas sur base du contexte fourni." si l'information n'est pas clairement présente

Paramètres dynamiques : `{question}` (question utilisateur) et `{context}` (texte concaténé des chunks avec pages).

Nous complétons ensuite la chaîne RAG : formatage des documents -> injection dans le prompt -> LLM Mistral -> sortie texte.


In [69]:
from langchain_core.prompts import PromptTemplate

assistant_prompt = PromptTemplate.from_template(
    """Tu es un Assistant Scolaire ECAM répondant aux questions sur le règlement interne.

CONTRAINTES:
1. Utilise UNIQUEMENT le contexte fourni qui vient du réglement interne interne.
2. Cite chaque fait avec la page sous forme [Page X].
3. Ne fabrique rien.

Format attendu:
Réponse concise en français.
CITES: Page: X,Y,... (liste unique de pages utilisées)

Question: {question}

Contexte:
{context}
"""
)

# Formateur des documents -> chaîne de caractères avec pages
# (La fonction format_with_metadata existe déjà plus haut; ici nous fournissons une version newline pour le prompt)
def format_docs(docs):
    return "\n".join(f"[Page {d.metadata.get('page','N/A')}] {d.page_content}" for d in docs)


In [ ]:
# Construction de la chaîne RAG finale (prompt -> LLM)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

# Sélecteur du retriever (assume ensemble déjà créé plus haut)
try:
    ensemble
except NameError:  # fallback au cas où
    similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})

llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

rag_chain = (
    {"context": similarity_retriever | format_docs, "question": RunnablePassthrough()} 
    | assistant_prompt
    | llm
    | StrOutputParser()
)

def ask(question: str) -> str:
    """Interroger l'assistant scolaire avec la question utilisateur."""
    return rag_chain.invoke(question)

# Test rapide (ajustez la question selon votre PDF)
print(ask("Quelles sont les sanctions pour fraude aux examens?"))
print(ask("Qu'est-ce qu'une AA?"))

Les sanctions pour fraude aux examens incluent :
- Une exclusion immédiate de la Haute École jusqu’au terme de l’année académique, prononcée par la Direction du Département [Page 42, 43].
- Un signalement au Commissaire du Gouvernement [Page 42, 43].
- Possibilité de recours interne (article 103 §2) et externe (article 104 §5) [Page 43].

CITES: Page: 42, 43.
Une AA (Activité d'Apprentissage) n'est pas explicitement définie dans le contexte fourni. Le règlement interne mentionne uniquement les "unités d'enseignement" (UE) et leurs modalités d'évaluation.

CITES: Page: 16, 31, 38
Une AA (Activité d'Apprentissage) n'est pas explicitement définie dans le contexte fourni. Le règlement interne mentionne uniquement les "unités d'enseignement" (UE) et leurs modalités d'évaluation.

CITES: Page: 16, 31, 38


In [72]:
# Section de test utilisateur : exécuter pour voir plusieurs réponses
try:
    ask  # vérifier que la fonction est définie
except NameError:
    raise RuntimeError("La fonction ask() n'est pas définie. Exécutez la cellule de construction de chaîne RAG avant.")

test_questions = [
    "Quelle est la procédure disciplinaire pour absentéisme?",
    "Quels sont les droits des étudiants?",
    "Quelles sont les règles sur le plagiat?",
    "Existe-t-il une règle sur l'usage des téléphones en classe?",
    "Comment sont traitées les fraudes aux examens?"
]
for q in test_questions:
    print("==="*12)
    print("Q:", q)
    print("A:", ask(q))

# Question ad-hoc (modifiez librement)
print("==="*12)
print("Q: Quelles sanctions sont prévues pour fraude aux examens?")
print("A:", ask("Quelles sanctions sont prévues pour fraude aux examens?"))

Q: Quelle est la procédure disciplinaire pour absentéisme?
A: En cas d'absentéisme, l'étudiante ou l'étudiant peut être sanctionné pédagogiquement (note d'échec) ou disciplinairement (selon l'article 100). La procédure disciplinaire inclut une audition préalable par l'autorité compétente, avec possibilité de se faire accompagner. Les sanctions possibles sont le blâme, des tâches d'intérêt collectif ou l'exclusion du stage.

CITES: Page: 15, 49, 50
Q: Quels sont les droits des étudiants?
A: En cas d'absentéisme, l'étudiante ou l'étudiant peut être sanctionné pédagogiquement (note d'échec) ou disciplinairement (selon l'article 100). La procédure disciplinaire inclut une audition préalable par l'autorité compétente, avec possibilité de se faire accompagner. Les sanctions possibles sont le blâme, des tâches d'intérêt collectif ou l'exclusion du stage.

CITES: Page: 15, 49, 50
Q: Quels sont les droits des étudiants?
A: Les droits des étudiants incluent :
- Recevoir les documents attestant l